# Datensätze miteinander verbinden

In [ ]:
import pandas as pd

Schauen wir uns einen Ausschnitt aus einer Umfrage bei allen Schulleiter*innen in NRW an, die gerade im WDR durchgeführt wird. 

In [ ]:
df = pd.read_csv('./data/umfrage_schulleiter.csv')

In [ ]:
df

## Wo vergeben Schulleiter*innen in NRW die besten Noten für die technische Ausstattung an ihrer Schule?
Unter f1_ausstattung haben die Schulleiter*innen Schulnoten für die Ausstattung ihrer Schule vergeben.
Im Datensatz gibt es nur die Spalte 'Ort', die einen Gemeindenamen enthält. 
Um die Daten nach Region analysieren und auch später visualisieren zu können, wäre es hilfreich, weitere Angaben wie den AGS, Regierungsbezirk etc. zu haben.

### Allgemeine NRW-Daten einladen

In [ ]:
# Laden der generellen Daten zu den NRW-Gemeinden 
city_data = pd.read_json('./data/nrw_data.json', orient='columns', dtype={'keyCity': 'str'})

In [ ]:
city_data.sample()

KeyCity enthält in diesem Fall den AGS ohne führende Null.

Wir wollen aus dem AGS die Regierungsbezirke-Namen ablesen, dafür erzeigen wir eine neue Spalte 'regierungsbezirk', die die erste Stelle aus dem AGS enthält.

In [ ]:
city_data['regierungsbezirk'] = city_data.keyCity.str[1:2]

In [ ]:
city_data.sample()

In [ ]:
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('1', 'Düsseldorf')
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('3', 'Köln')
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('5', 'Münster')
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('7', 'Detmold')
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('9', 'Arnsberg')

In [ ]:
city_data.regierungsbezirk.value_counts()

### Datensätze verbinden

In [ ]:
# Um die Datensätze zu verbinden, setzen wir in beiden Fällen den Index auf die Ort-Spalte
df = df.set_index('Ort')

In [ ]:
df.sample()

In [ ]:
city_data = city_data.set_index('city')

In [ ]:
city_data.sample()

In [ ]:
# Um zu kontrollieren, dass keine Spalten verloren gehen, überprüfen wir die Länge der beiden Datensätze
len(df), len(city_data)

In [ ]:
# Join verbindet beide Datensätze anhand des index
df = df.join(city_data)

In [ ]:
len(df)

In [ ]:
df.sample(10)

#### Tipp: 

Es gibt noch weitere Arten, Datensätze miteinander zu verknüpfen, einen ersten Überblick findest du hier:
        [Chris Albon: Join And Merge Pandas Dataframe](https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/)

### Wie bewerten Sie die digitale Ausstattung Ihrer Schule zurzeit?

In [ ]:
# Gibt es Schulen, die diese Frage nicht beantwortet haben
df.F1_ausstattung.value_counts(dropna=False)

In [ ]:
df[df.F1_ausstattung.isna()]

## Durchschnitt aller Schulen

In [ ]:
# Pandas hat eine eigene Funktion, um Durchschnoittswerte zu bilden
df.F1_ausstattung.mean()

### Zufriedenheit nach Regierungsbezirken 

In [ ]:
# Um zu ermitteln, in welchem Regierungsbezirk die Schulleiter*innen am zufriedensten sind, nutzen wir group_by: 
df.groupby('regierungsbezirk').mean()

### Übung

Schreibe die jeweilige group_by - Funktion

In [ ]:
# Zufriedenheit nach Stadt / Land (type)


In [ ]:
# Zufriedenheit nach Regionen


In [ ]:
Du kannst die Daten auch sortieren

In [ ]:
# Zufriedenheit nach Landkreisen, sortiert nach Ausstattung
df.groupby('district').mean().sort_values(by='F1_ausstattung', ascending=False)

### Übung

In [ ]:
# Nach Schulform, sortiert nach Ausstattung


Wo gibt es aus deiner Sicht die spannendsten Ergebnisse?

### Übung

Wie findest Du heraus, wie viele Schulen eine 1 bzw. eine 6 vergeben haben?

In [ ]:
# Wieviele Schulen sind super zufrieden (1.0)


In [ ]:
# Wieviele Schulen sind super unzufrieden (6.0)


Auf zur nächsten Datenquelle: 
    
**[ >> Excel-Dokumente Notebook](06b_excel.ipynb)**